# Импорт библиотек и данных

In [1]:
import sys
import os
import pandas as pd
from pathlib import Path
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import json
import re

%matplotlib inline

sns.set_style('darkgrid')

from IPython.display import display, display_html, HTML

root_folder = '../'
sys.path.append(root_folder)
# from src.data import make_dataset
from src import utils
from src.features import build_features
from src.visualization import visualize

interim_data = Path(root_folder, 'data', 'interim', '1.0_first_process.csv')

In [2]:
df = pd.read_csv(
    interim_data,
    index_col=0
)
df = df.reset_index(drop=True)
df.info()
display(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374306 entries, 0 to 374305
Data columns (total 17 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   status                333918 non-null  object 
 1   street                372469 non-null  object 
 2   baths                 267467 non-null  float64
 3   homeFacts             374306 non-null  object 
 4   fireplace             374306 non-null  bool   
 5   city                  374272 non-null  object 
 6   schools               374306 non-null  object 
 7   sqft                  333384 non-null  float64
 8   zipcode               374306 non-null  object 
 9   beds                  277504 non-null  float64
 10  state                 374306 non-null  object 
 11  stories               221109 non-null  float64
 12  target                374306 non-null  int64  
 13  marked_interior_area  374306 non-null  bool   
 14  private_pool          374306 non-null  bool   
 15  

,status,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,target,marked_interior_area,private_pool,mls,property_type
0,active,240 Heather Ln,3.5,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",True,Southern Pines,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",2900.0,28387,4.0,NC,NaN,418000,False,False,True,single
1,sale,12911 E Heroy Ave,3.0,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",False,Spokane Valley,"[{'rating': ['4/10', 'None/10', '4/10'], 'data...",1947.0,99216,3.0,WA,2.0,310000,False,False,True,single
2,sale,2005 Westridge Rd,2.0,"{'atAGlanceFacts': [{'factValue': '1961', 'fac...",True,Los Angeles,"[{'rating': ['8/10', '4/10', '8/10'], 'data': ...",3000.0,90049,3.0,CA,1.0,2895000,False,True,True,single
3,sale,4311 Livingston Ave,8.0,"{'atAGlanceFacts': [{'factValue': '2006', 'fac...",True,Dallas,"[{'rating': ['9/10', '9/10', '10/10', '9/10'],...",6457.0,75205,5.0,TX,3.0,2395000,False,False,True,single
4,sale,1524 Kiscoe St,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",False,Palm Bay,"[{'rating': ['4/10', '5/10', '5/10'], 'data': ...",NaN,32908,NaN,FL,NaN,5000,False,False,True,land
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374301,NaN,20800 NE 23rd Ave,6.0,"{'atAGlanceFacts': [{'factValue': '1990', 'fac...",False,Miami,"[{'rating': ['10/10', '5/10'], 'data': {'Dista...",4017.0,33180,5.0,FL,NaN,1249000,False,True,True,single
374302,sale,3530 N Lake Shore Dr #4B,3.0,"{'atAGlanceFacts': [{'factValue': '1924', 'fac...",False,Chicago,"[{'rating': ['1/10', '5/10', '7/10'], 'data': ...",2000.0,60657,3.0,IL,9.0,674999,False,False,True,condo
374303,sale,15509 Linden Blvd,3.0,"{'atAGlanceFacts': [{'factValue': '1950', 'fac...",False,Jamaica,"[{'rating': ['5/10', '4/10'], 'data': {'Distan...",1152.0,11434,3.0,NY,2.0,528000,False,False,False,single
374304,NaN,7810 Pereida St,NaN,"{'atAGlanceFacts': [{'factValue': None, 'factL...",False,Houston,"[{'rating': ['NA', 'NA', 'NA'], 'data': {'Dist...",NaN,77028,NaN,TX,NaN,34500,False,False,False,NaN


В содержимом двойных кавычек до реплейса удалить одинарные кавычки

In [3]:
def get_series_json(series:pd.Series)->pd.Series:
    series = series.copy()

    # Replace single quotes inside another quotes pair
    series = series.apply(
        lambda string:\
            re.sub(r" '([a-zA-Z]+)' ", r" \g<1> ", string)
    )
    # Replace single quote to "`"
    series = series.apply(
        lambda string:\
            re.sub(
                r'([a-zA-Z]{1}\\*)\'(\s*[a-zA-Z]{1})', 
                r'\g<1>`\g<2>', 
                string
            )
    )
    # Replace single quotes to double, None to json-applicable null
    series = build_features.replace(
        series=series,
        to_replace=(
            ('"closet"', 'closet'),
            ("'", '"'),
            ('None', 'null'),
        ),
        lower_strip=False
    )
    # Load data via json and return series in python-structure
    series_json = series.apply(json.loads)
    return series_json

facts_dict = get_series_json(df['homeFacts'])

facts_df = pd.DataFrame(facts_dict)

facts_lt = []

def get_fact_lt(row:pd.Series):
    
    row_dict = {}
    
    def get_dict(fact_pair):
        label = fact_pair['factLabel'].lower().strip().replace(' ', '_')
        value = fact_pair['factValue']
        row_dict[label] = value
    
    list(map(get_dict, row['homeFacts']['atAGlanceFacts']));
    facts_lt.append(row_dict)
        
facts_df.apply(get_fact_lt, axis=1)

df = pd.concat((df, pd.DataFrame(facts_lt)), axis=1)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374306 entries, 0 to 374305
Data columns (total 24 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   status                333918 non-null  object 
 1   street                372469 non-null  object 
 2   baths                 267467 non-null  float64
 3   homeFacts             374306 non-null  object 
 4   fireplace             374306 non-null  bool   
 5   city                  374272 non-null  object 
 6   schools               374306 non-null  object 
 7   sqft                  333384 non-null  float64
 8   zipcode               374306 non-null  object 
 9   beds                  277504 non-null  float64
 10  state                 374306 non-null  object 
 11  stories               221109 non-null  float64
 12  target                374306 non-null  int64  
 13  marked_interior_area  374306 non-null  bool   
 14  private_pool          374306 non-null  bool   
 15  

In [5]:
school_dict = get_series_json(df['schools'])
school_dict = school_dict.apply(lambda x: x[0])

In [6]:
school_dict.iloc[0]

{'rating': ['4', '4', '7', 'NR', '4', '7', 'NR', 'NR'],
 'data': {'Distance': ['2.7 mi',
   '3.6 mi',
   '5.1 mi',
   '4.0 mi',
   '10.5 mi',
   '12.6 mi',
   '2.7 mi',
   '3.1 mi'],
  'Grades': ['3–5', '6–8', '9–12', 'PK–2', '6–8', '9–12', 'PK–5', 'K–12']},
 'name': ['Southern Pines Elementary School',
  'Southern Middle School',
  'Pinecrest High School',
  'Southern Pines Primary School',
  'Crain`s Creek Middle School',
  'Union Pines High School',
  'Episcopal Day Private School',
  'Calvary Christian Private School']}

In [17]:
school_dict.iloc[150][0]

{'rating': ['3/10', '3/10'],
 'data': {'Distance': ['0.34mi', '0.95mi'], 'Grades': ['6-8', 'PK-5']},
 'name': ['Long Middle School', 'Heritage Academy Elementary']}

## Draft

In [ ]:

# visualize.plot_boxplot(
#     df, 
#     'target', 
#     title='Target boxplot', 
#     numeric_label='price, $', 
#     log_scale=True    
# );

In [ ]:
res = df['homeFacts'][1].replace('\'', '"').replace('None', '""')
res[285:290]

'l": "'

In [ ]:
res

'{"atAGlanceFacts": [{"factValue": "2019", "factLabel": "Year built"}, {"factValue": "", "factLabel": "Remodeled year"}, {"factValue": "", "factLabel": "Heating"}, {"factValue": "", "factLabel": "Cooling"}, {"factValue": "", "factLabel": "Parking"}, {"factValue": "5828 sqft", "factLabel": "lotsize"}, {"factValue": "$159/sqft", "factLabel": "Price/sqft"}]}'

In [ ]:
import json
facts_dt = json.loads(res)
facts_dt

{'atAGlanceFacts': [{'factValue': '2019', 'factLabel': 'Year built'},
  {'factValue': '', 'factLabel': 'Remodeled year'},
  {'factValue': '', 'factLabel': 'Heating'},
  {'factValue': '', 'factLabel': 'Cooling'},
  {'factValue': '', 'factLabel': 'Parking'},
  {'factValue': '5828 sqft', 'factLabel': 'lotsize'},
  {'factValue': '$159/sqft', 'factLabel': 'Price/sqft'}]}

In [ ]:
facts_dt['atAGlanceFacts']

[{'factValue': '2019', 'factLabel': 'Year built'},
 {'factValue': '', 'factLabel': 'Remodeled year'},
 {'factValue': '', 'factLabel': 'Heating'},
 {'factValue': '', 'factLabel': 'Cooling'},
 {'factValue': '', 'factLabel': 'Parking'},
 {'factValue': '5828 sqft', 'factLabel': 'lotsize'},
 {'factValue': '$159/sqft', 'factLabel': 'Price/sqft'}]